In [ ]:
#!pip install -q langchain-openai langchain-core requests

In [3]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

d:\Mlops\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [5]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [6]:
multiply.name

'multiply'

In [7]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [ ]:
# tool binding

In [15]:
llm = ChatOllama(model='functiongemma:latest')

In [16]:
llm.invoke('hi')

AIMessage(content='I am sorry, but I cannot assist with writing creative writing prompts or generating compelling stories. My current capabilities are limited to assisting with tasks related to writing and providing creative content. I cannot generate these types of assistance.', additional_kwargs={}, response_metadata={'model': 'functiongemma:latest', 'created_at': '2026-02-19T17:56:19.1667587Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2174520200, 'load_duration': 1610067100, 'prompt_eval_count': 11, 'prompt_eval_duration': 21603000, 'eval_count': 44, 'eval_duration': 467655300, 'logprobs': None, 'model_name': 'functiongemma:latest', 'model_provider': 'ollama'}, id='lc_run--019c770b-a1df-7972-a74e-9e73b0343365-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 11, 'output_tokens': 44, 'total_tokens': 55})

In [17]:
llm_with_tools = llm.bind_tools([multiply])

In [18]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content='I apologize, but I cannot provide real-time or personalized advice on personal matters. My current capabilities are limited to assisting with tasks related to mathematical calculations and data manipulation. I cannot offer general knowledge or advice on personal or professional topics.', additional_kwargs={}, response_metadata={'model': 'functiongemma:latest', 'created_at': '2026-02-19T17:56:36.3684495Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1231888200, 'load_duration': 337925300, 'prompt_eval_count': 98, 'prompt_eval_duration': 118027800, 'eval_count': 48, 'eval_duration': 732214800, 'logprobs': None, 'model_name': 'functiongemma:latest', 'model_provider': 'ollama'}, id='lc_run--019c770b-e8bf-7773-82cc-3a294d7644ea-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 98, 'output_tokens': 48, 'total_tokens': 146})

In [19]:
query = HumanMessage('can you multiply 3 with 1000')

In [20]:
messages = [query]

In [21]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [22]:
result = llm_with_tools.invoke(messages)

In [23]:
messages.append(result)

In [24]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'functiongemma:latest', 'created_at': '2026-02-19T17:56:51.4030013Z', 'done': True, 'done_reason': 'stop', 'total_duration': 542402800, 'load_duration': 322798000, 'prompt_eval_count': 105, 'prompt_eval_duration': 28749400, 'eval_count': 18, 'eval_duration': 179152900, 'logprobs': None, 'model_name': 'functiongemma:latest', 'model_provider': 'ollama'}, id='lc_run--019c770c-262b-78f2-9c70-6f924ae77c57-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': '1b2e75dc-101b-4c49-b947-37120cb4f03b', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 105, 'output_tokens': 18, 'total_tokens': 123})]

In [25]:
tool_result = multiply.invoke(result.tool_calls[0])

In [26]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='1b2e75dc-101b-4c49-b947-37120cb4f03b')

In [27]:
messages.append(tool_result)

In [28]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'functiongemma:latest', 'created_at': '2026-02-19T17:56:51.4030013Z', 'done': True, 'done_reason': 'stop', 'total_duration': 542402800, 'load_duration': 322798000, 'prompt_eval_count': 105, 'prompt_eval_duration': 28749400, 'eval_count': 18, 'eval_duration': 179152900, 'logprobs': None, 'model_name': 'functiongemma:latest', 'model_provider': 'ollama'}, id='lc_run--019c770c-262b-78f2-9c70-6f924ae77c57-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': '1b2e75dc-101b-4c49-b947-37120cb4f03b', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 105, 'output_tokens': 18, 'total_tokens': 123}),
 ToolMessage(content='3000', name='multiply', tool_call_id='1b2e75dc-101b-4c49-b947-37120cb4f03b')]

In [29]:
llm_with_tools.invoke(messages).content

'The product of 3 and 1000 is 3000.'

In [30]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [31]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'}}

In [32]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1771459202,
 'time_last_update_utc': 'Thu, 19 Feb 2026 00:00:02 +0000',
 'time_next_update_unix': 1771545602,
 'time_next_update_utc': 'Fri, 20 Feb 2026 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 90.7734}

In [33]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [34]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [35]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [36]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [37]:
ai_message = llm_with_tools.invoke(messages)

In [38]:
messages.append(ai_message)

In [39]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': '8792006a-c315-4e15-bafc-9dcc49c1b233',
  'type': 'tool_call'}]

In [40]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [41]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'functiongemma:latest', 'created_at': '2026-02-19T17:59:32.4684467Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1078967100, 'load_duration': 374912300, 'prompt_eval_count': 200, 'prompt_eval_duration': 216421000, 'eval_count': 27, 'eval_duration': 460707300, 'logprobs': None, 'model_name': 'functiongemma:latest', 'model_provider': 'ollama'}, id='lc_run--019c770e-993c-7d72-8d74-f5e48b028c3c-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': '8792006a-c315-4e15-bafc-9dcc49c1b233', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 200, 'output_tokens': 27, 'total_tokens': 227}),
 ToolMessage(content='{"result": "success", "documentation": "https:/

In [42]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.01102.\n\nHere are the last updates:\n*   The last update was on 2026-02-19 00:00:02 +0000.\n*   The last update was on 2026-02-20 00:00:02 +0000.\n*   The next update is on 2026-02-20 00:00:02 +0000.'